In [1]:
from scrappers.scrappers import combine_data
#from config import standard_url
from scrappers.scrappers import get_player_links
import pandas as pd
years = ["2024-2025"]
import os
standard_url = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
]

In [2]:
def make_matchlog_links(player_urls, years, stats):
    links = {stat: [] for stat in stats}  # dict of lists
    for url in player_urls:
        parts = url.strip("/").split("/")
        player_id = parts[-2]
        player_slug = parts[-1]
        for year in years:
            for stat in stats:
                matchlog_url = f"https://fbref.com/en/players/{player_id}/matchlogs/{year}/{stat}/{player_slug}-Match-Logs"
                links[stat].append({
                    "name": f"{player_slug} - {stat}",
                    "url": matchlog_url
                })
    return links


In [5]:
def get_match_logs(years, player_links, stats):
    matchlog_links = make_matchlog_links(player_links, years, stats)
    
    all_dfs = {}
    for stat, links in matchlog_links.items():
        dfs = []
        for link in links:
            try:
                df_list = combine_data(years, [link], table_id="matchlogs_all")  # may return a list
                if df_list:  
                    # flatten if df_list is already a list of DataFrames
                    if isinstance(df_list, list):
                        dfs.extend(df_list)
                    else:
                        dfs.append(df_list)
            except Exception as e:
                print(f"Error retrieving {link['url']}: {e}")
        if dfs:
            all_dfs[stat] = pd.concat(dfs, ignore_index=True)
        else:
            print(f"No data retrieved for category {stat}")
    return all_dfs


In [7]:
stats = ["summary","defense","passing","gca","possession","passing_types","keeper"]
player_links = get_player_links(years, standard_url)

player_links = player_links[:2]

all_match_logs = get_match_logs(years, player_links, stats)

# Save each category separately
os.makedirs("data/raw", exist_ok=True)
for stat, df in all_match_logs.items():
    output_path = os.path.join("data", "raw", f"player_match_logs_{stat}.csv")
    df.to_csv(output_path, index=False)
    print(f"Saved {stat} match logs to {output_path}")


found stats_standard for https://fbref.com/en/comps/20/2024-2025/stats/2024-2025-Bundesliga-Stats
200 https://fbref.com/en/players/82e12fe7/matchlogs/2024-2025/summary/Junior-Adamu-Match-Logs
200 https://fbref.com/en/players/501055d8/matchlogs/2024-2025/summary/Karim-Adeyemi-Match-Logs
200 https://fbref.com/en/players/82e12fe7/matchlogs/2024-2025/defense/Junior-Adamu-Match-Logs
200 https://fbref.com/en/players/501055d8/matchlogs/2024-2025/defense/Karim-Adeyemi-Match-Logs
200 https://fbref.com/en/players/82e12fe7/matchlogs/2024-2025/passing/Junior-Adamu-Match-Logs
200 https://fbref.com/en/players/501055d8/matchlogs/2024-2025/passing/Karim-Adeyemi-Match-Logs
200 https://fbref.com/en/players/82e12fe7/matchlogs/2024-2025/gca/Junior-Adamu-Match-Logs
200 https://fbref.com/en/players/501055d8/matchlogs/2024-2025/gca/Karim-Adeyemi-Match-Logs
200 https://fbref.com/en/players/82e12fe7/matchlogs/2024-2025/possession/Junior-Adamu-Match-Logs
200 https://fbref.com/en/players/501055d8/matchlogs/2024-